# Analiza danych w języku Python - ćwiczenia laboratoryjne 2022/2023

Ten notatnik zalicza się do grupy zestawów zadań, na podstawie których odbywa się zaliczenie ćwiczeń i podlega zwrotowi do oceny w ustalonym na zajęciach terminie.

Uwagi i wytyczne ogólne dotyczące uzupełniania i oceny notatnika:
- Podczas wykonywania zadań należy korzystać wyłącznie z pakietów zaimportowanych na początku notatnika oraz z pakietów wchodzących w skład standardowej biblioteki Pythona, które można zaimportować samodzielnie we wskazanej komórce notatnika.
- Swoje rozwiązania należy wprowadzać wyłącznie w miejce następujących fragmentów kodu:<br/> `# YOUR CODE HERE`<br/> `raise NotImplementedError()`<br/> 
a odpowiedzi tekstowe w komórkach oznaczonych hasłem:<br/> 
`YOUR ANSWER HERE`<br/> 
Nie należy w żaden sposób modyfikować pozostałych fragmentów kodu oraz innych elementów notatnika, w szczególności dodawać lub usuwać komórek oraz zmieniać nazwy pliku.
- Jeżeli zestaw zadań wymaga skorzystania z fragmentów kodu opracowanego w ramach wcześniejszych zestawów zadań należy je umieścić we wskazanej komórce notatnika.
- Otrzymywane wyniki i odpowiedzi mają być rezultatem wykonania napisanego kodu, odpowiedzi uzupełniane manualnie nie podlegają ocenie.
- Zadanie należy wykonać w taki sposób, aby podczas wykonywania kodu nie zostały wyświetlone żadne ostrzeżenia.
- Zawarte w notatniku automatyczne testy mają charakter poglądowy. Dotyczą one wybranych aspektów zadań i mają za zadanie wyłapać podstawowe błędy. Przejście wszystkich testów nie oznacza, że zadanie jest wykonane w całości poprawnie i zostanie ocenione na maksymalną liczbę punktów.
- Zadania, które powodują wyświetlenie komunikatu o błędzie przerywającym wykonywanie kodu nie podlegają ocenie.

Uwagi i wytyczne ogólne dotyczące wizualizacji wyników:
- Wszystkie wykresy powinny być wykonane w jednolitym, przejrzystym i czytelnym stylu, posiadać odpowiednio dobrane proporcje i zakresy wartości osi.
- Wykresy oraz ich osie powinny mieć nadane tytuły. Jeżeli w obrębie figury znajduje się więcej niż jeden wykres to figura również powinna mieć nadany tytuł. 
- Figury powinny mieć ustawione białe tło, tak, aby niezależnie od ustawień notatnika wszystkie elementy wykresów były dobrze widoczne (domyślnie tło jest przeźroczyste co może powodować problemy w notatnikach z ustawionym ciemnym tłem).
- Rozmiar poziomy figur nie powinien przekraczać 20 cali.

Przed odesłaniem zestawu zadań do oceny proszę uzupełnić komórkę z danymi autorów rozwiązania (`NAME` - nazwa zespołu, `COLLABORATORS` - imiona, nazwiska i numery indeksów członków zespołu) oraz upewnić się, że notatnik działa zgodnie z oczekiwaniami. W tym celu należy skorzystać z opcji **Restart Kernel and Run All Cells...** dostępnej na górnej belce notatnika pod symbolem $\blacktriangleright\blacktriangleright$.

In [1]:
NAME = "IAD28"
COLLABORATORS = "Radosław Matuszczyk 405124, Piotr Łukasiewicz 406163"

---

## Zestaw zadań 4: Wybrane teoretyczne rozkłady prawdopodobieństwa oraz elementy teorii estymacji (Notatnik 2/3)

In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów
def variance_estimator(data, population_mean="unknown", data_type="raw_scores"):
    N = len(data['x']) if data_type == "raw_scores" else data['f'].sum()
    biased = population_mean != "unknown"
    
    S = variance_sum_estimator(data, population_mean, data_type)
    
    if biased:
        S /= N
    else:
        S /= N-1
    
    return S

def variance_sum_estimator(data, population_mean="unknown", data_type="raw_scores"):
    if population_mean == "unknown":
        population_mean = mean_estimator(data, data_type)
    
    if data_type == "raw_scores":
        return ((data['x'] - population_mean)**2).sum()
    
    if data_type == "simple_frequency_distribution":
        return (data.index - population_mean)**2 @ data['f']
    
    if data_type == "grouped_frequency_distribution":
        return (data.index.mid - population_mean)**2 @ data['f']
    
def mean_estimator(data, data_type="raw_scores"):
    if data_type == "raw_scores":
        return data.values.mean()
    elif data_type == 'simple_frequency_distribution':
        return (data.index @ data['f']) / data['f'].sum()
    else:
        return data.index.mid @ data['f'] / data['f'].sum()
    

In [4]:
# Dane do testów dla szeregów szczegółowych
test_data_1 = pd.DataFrame(data=sp.stats.norm.rvs(loc=5, scale=0.2, size=20, random_state=7), columns=["x"])
test_data_1

,x
0,5.338105
1,4.906813
2,5.006564
3,5.081503
4,4.842215
5,5.000413
6,4.999822
7,4.649055
8,5.203532
9,5.120100


In [5]:
# Dane do testów dla szeregów rozdzielczych
test_population = pd.DataFrame(data=sp.special.round(sp.stats.norm.rvs(loc=10, scale=3, size=10000, random_state=3)), columns=["x"])
raw_test_data_2 = test_population.sample(50, random_state=21)
test_data_2 = raw_test_data_2.groupby('x').size().reset_index(name='f').rename(columns={'Col1':'Col_value'})
test_data_2.set_index('x', inplace=True)
test_data_2

,f
x,
4.0,3
5.0,1
6.0,1
7.0,7
8.0,7
9.0,5
10.0,8
11.0,5
12.0,2


In [6]:
# Dane do testów dla szeregów rozdzielczych przedziałowych
raw_test_data_3 = pd.DataFrame(data=sp.stats.norm.rvs(loc=-5, scale=2, size=100, random_state=25), columns=["x"])
bins = np.arange(-11, 2, 1) 
index = pd.Index(pd.cut(raw_test_data_3['x'], bins).cat.categories)
test_data_3 = pd.DataFrame(data=raw_test_data_3.groupby(pd.cut(raw_test_data_3['x'], bins)).count().values, index=index)
test_data_3.index.rename("x", inplace=True)
test_data_3.rename(columns= {0: "f"}, inplace=True)
test_data_3

,f
x,
"(-11, -10]",1
"(-10, -9]",2
"(-9, -8]",7
"(-8, -7]",7
"(-7, -6]",20
"(-6, -5]",16
"(-5, -4]",21
"(-4, -3]",8
"(-3, -2]",8


### Zadanie 3: Estymacja przedziałowa wartości oczekiwanej [15 pkt]

Przygotuj funkcję `interval_mean_estimation()`, która będzie dokonywała estymacji przedziałowej wartości oczekiwanej dla trzech rodzajów danych wejściowych - szeregu szczegółowego, szeregu rozdzielczego i szeregu rozdzielczego przedziałowego.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`𝜇 należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [7]:
def interval_mean_estimation(data, alpha, population_std="unknown", data_type="raw_scores"):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca w zależności od wartości parametru data_type:
        - domyślny indeks i pojedynczą kolumnę "x" z wartościami próby losowej,
        - indeks o nazwie "x" z wariantami cechy oraz pojedynczą kolumnę "f" ze zliczeniami wariantów,
        - indeks (IntervalIndex) o nazwie "x" z przedziałami wartości cechy oraz pojedynczą kolumnę "f" ze zliczeniami wariantów w poszczególnych przedziałach.
    alpha: float
        Wartość określająca poziom istotności.
    population_std: str or float
        Jeżeli odchylenie standardowe populacji nie jest znane to parametr przyjmuje wartość "unknown",
        w przeciwnym wypadku jest to wartość określająca odchylenie standardowe populacji.
    data_type: str
        Określa typ szeregu statystycznego:
        - "raw_scores" - szereg szczegółowy,
        - "simple_frequency_distribution" - szereg rozdzielczy,
        - "grouped_frequency_distribution" - szereg rozdzielczy przedziałowy.
    
    Returns
    -------
    mean_low: float
        Dolna granica wyliczonego przedziału ufności.
    mean_high: float
        Górna granica wyliczonego przedziału ufności.
    """
    data_mean = mean_estimator(data, data_type)
    n = len(data['x']) if data_type == "raw_scores" else data['f'].sum()
    n_sqrt = np.sqrt(n)
    if population_std != 'unknown':
        u = sp.stats.norm.ppf(1-alpha/2)
        mean_low = data_mean - u*population_std/n_sqrt
        mean_high = data_mean + u*population_std/n_sqrt
    else:
        t = sp.stats.t.ppf(1-alpha/2, n-1)
        S = variance_estimator(data, 'unknown', data_type)
        mean_low = data_mean - t*np.sqrt(S)/n_sqrt
        mean_high = data_mean + t*np.sqrt(S)/n_sqrt
    print("𝜇 należy do przedziału [{}, {}] przy założeniu poziomu ufności 1-𝛼 = {}".format(mean_low, mean_high, 1-alpha))
    return [mean_low, mean_high]

In [8]:
### Komórka testowa

# Testy dla szeregów szczegółowych

# Test na test_data_1 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_mean_estimation(test_data_1, 0.1, population_std=0.2, data_type="raw_scores"), (4.905619719495418, 5.05273990041144))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_1, 0.05, population_std=0.2, data_type="raw_scores"), (4.891527555895771, 5.066832064011087))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_1, 0.02, population_std=0.2, data_type="raw_scores"), (4.875142370240081, 5.083217249666777))))
print()

# # Test na test_data_1 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_mean_estimation(test_data_1, 0.1, data_type="raw_scores"), (4.913356369231727, 5.045003250675131))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_1, 0.05, data_type="raw_scores"), (4.899504007229391, 5.058855612677467))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_1, 0.02, data_type="raw_scores"), (4.882508507351398, 5.07585111255546))))

𝜇 należy do przedziału [4.905619719495418, 5.05273990041144] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜇 należy do przedziału [4.891527555895771, 5.066832064011087] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜇 należy do przedziału [4.875142370240081, 5.083217249666777] przy założeniu poziomu ufności 1-𝛼 = 0.98

𝜇 należy do przedziału [4.913356369231727, 5.045003250675131] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜇 należy do przedziału [4.899504007229391, 5.058855612677467] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜇 należy do przedziału [4.882508507351398, 5.07585111255546] przy założeniu poziomu ufności 1-𝛼 = 0.98


In [9]:
### Komórka testowa

# Testy dla szeregów rozdzielczych

# Test na test_data_2 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_mean_estimation(test_data_2, 0.1, population_std=3, data_type="simple_frequency_distribution"), (9.002147707793995, 10.397852292206004))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_2, 0.05, population_std=3, data_type="simple_frequency_distribution"), (8.868457705390192, 10.531542294609807))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_2, 0.02, population_std=3, data_type="simple_frequency_distribution"), (8.713014185720088, 10.686985814279911))))
print()

# Test na test_data_2 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_mean_estimation(test_data_2, 0.1, data_type="simple_frequency_distribution"), (8.985882655267286, 10.414117344732713))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_2, 0.05, data_type="simple_frequency_distribution"), (8.844032825120204, 10.555967174879795))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_2, 0.02, data_type="simple_frequency_distribution"), (8.675649993053778, 10.72435000694622))))

𝜇 należy do przedziału [9.002147707793995, 10.397852292206004] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜇 należy do przedziału [8.868457705390192, 10.531542294609807] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜇 należy do przedziału [8.713014185720088, 10.686985814279911] przy założeniu poziomu ufności 1-𝛼 = 0.98

𝜇 należy do przedziału [8.985882655267286, 10.414117344732713] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜇 należy do przedziału [8.844032825120204, 10.555967174879795] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜇 należy do przedziału [8.675649993053778, 10.72435000694622] przy założeniu poziomu ufności 1-𝛼 = 0.98


In [10]:
### Komórka testowa

# Testy dla szeregów rozdzielczych przedziałowych

# Test na test_data_3 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_mean_estimation(test_data_3, 0.1, population_std=2, data_type="grouped_frequency_distribution"), (-5.448970725390295, -4.791029274609706))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_3, 0.05, population_std=2, data_type="grouped_frequency_distribution"), (-5.511992796908011, -4.728007203091989))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_3, 0.02, population_std=2, data_type="grouped_frequency_distribution"), (-5.585269574808168, -4.654730425191832))))
print()

# Test na test_data_3 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_mean_estimation(test_data_3, 0.1, data_type="grouped_frequency_distribution"), (-5.487717471820824, -4.752282528179176))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_3, 0.05, data_type="grouped_frequency_distribution"), (-5.5594333457618, -4.6805666542382))))
print()
assert(np.all(np.isclose(interval_mean_estimation(test_data_3, 0.02, data_type="grouped_frequency_distribution"), (-5.643675933878456, -4.596324066121544))))

𝜇 należy do przedziału [-5.448970725390295, -4.791029274609706] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜇 należy do przedziału [-5.511992796908011, -4.728007203091989] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜇 należy do przedziału [-5.585269574808168, -4.654730425191832] przy założeniu poziomu ufności 1-𝛼 = 0.98

𝜇 należy do przedziału [-5.487717471820824, -4.752282528179176] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜇 należy do przedziału [-5.5594333457618, -4.6805666542382] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜇 należy do przedziału [-5.643675933878456, -4.596324066121544] przy założeniu poziomu ufności 1-𝛼 = 0.98


### Zadanie 4: Estymacja przedziałowa wariancji [15 pkt]
Przygotuj funkcję `interval_variance_estimation()`, która będzie dokonywała estymacji przedziałowej wariancji dla trzech rodzajów danych wejściowych - szeregu szczegółowego, szeregu rozdzielczego i szeregu rozdzielczego przedziałowego.

Oprócz zwracanych wartości granic przedziału funkcja powinna wyświetlać następujący komunikat:

`𝜎^2  należy do przedziału [X, Y] przy założeniu poziomu ufności 1-𝛼 = Z`

gdzie X, Y i Z są automatycznie uzupełniane przez funkcję, a X i Y dodatkowo sformatowane w taki sposób, żeby wyświetlały się z dokładnością do 4 miejsc po przecinku.

In [11]:
def interval_variance_estimation(data, alpha, population_mean="unknown", data_type="raw_scores"):
    """
    Parameters
    -------
    data: DataFrame
        Tablica zawierająca w zależności od wartości parametru data_type:
        - domyślny indeks i pojedynczą kolumnę "x" z wartościami próby losowej,
        - indeks o nazwie "x" z wariantami cechy oraz pojedynczą kolumnę "f" ze zliczeniami wariantów,
        - indeks (IntervalIndex) o nazwie "x" z przedziałami wartości cechy oraz pojedynczą kolumnę "f" ze zliczeniami wariantów w poszczególnych przedziałach.
    alpha: float
        Wartość określająca poziom istotności.
    population_mean: str or float
        Jeżeli wartość oczekiwana populacji nie jest znane to parametr przyjmuje wartość "unknown",
        w przeciwnym wypadku jest to wartość określająca wartość oczekiwaną populacji.
    data_type: str
        Określa typ szeregu statystycznego:
        - "raw_scores" - szereg szczegółowy,
        - "simple_frequency_distribution" - szereg rozdzielczy,
        - "grouped_frequency_distribution" - szereg rozdzielczy przedziałowy.
    
    Returns
    -------
    var_low: float
        Dolna granica wyliczonego przedziału ufności.
    var_high: float
        Górna granica wyliczonego przedziału ufności.
    """
    N = len(data['x']) if data_type == "raw_scores" else data['f'].sum()

    S = variance_sum_estimator(data, population_mean, data_type)
    
    low = S / sp.stats.chi2.ppf(1-alpha/2, N-1)
    high = S / sp.stats.chi2.ppf(alpha/2, N-1)
    print(f'𝜎^2 należy do przedziału [{low :.4f}, {high :.4f}] przy założeniu poziomu ufności 1-𝛼 = {1 - alpha}')
    return (low, high)

In [12]:
### Komórka testowa

# Testy dla szeregów szczegółowych

# Test na test_data_1 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_variance_estimation(test_data_1, 0.1, population_mean=5, data_type="raw_scores"), (0.01855573624556403, 0.05528660848892888))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_1, 0.05, population_mean=5, data_type="raw_scores"), (0.017025745016269588, 0.06280068548350207))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_1, 0.02, population_mean=5, data_type="raw_scores"), (0.015455150810030724, 0.07328116757245808))));
print()

# Test na test_data_1 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_variance_estimation(test_data_1, 0.1, data_type="raw_scores"), (0.018268125369359626, 0.05442967510189219))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_1, 0.05, data_type="raw_scores"), (0.0167618487538223, 0.06182728513230527))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_1, 0.02, data_type="raw_scores"), (0.015215598500840796, 0.07214532146342346))));

𝜎^2 należy do przedziału [0.0186, 0.0553] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2 należy do przedziału [0.0170, 0.0628] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2 należy do przedziału [0.0155, 0.0733] przy założeniu poziomu ufności 1-𝛼 = 0.98

𝜎^2 należy do przedziału [0.0183, 0.0544] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2 należy do przedziału [0.0168, 0.0618] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2 należy do przedziału [0.0152, 0.0721] przy założeniu poziomu ufności 1-𝛼 = 0.98


In [13]:
### Komórka testowa

# Testy dla szeregów rozdzielczych

# Test na test_data_2 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_variance_estimation(test_data_2, 0.1, population_mean=10, data_type="simple_frequency_distribution"), (6.7683018526269425, 13.23300783223187))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_2, 0.05, population_mean=10, data_type="simple_frequency_distribution"), (6.393969919236964, 14.229161421841045))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_2, 0.02, population_mean=10, data_type="simple_frequency_distribution"), (5.993101315036716, 15.514512026199176))));
print()

# Test na test_data_2 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_variance_estimation(test_data_2, 0.1, data_type="simple_frequency_distribution"), (6.700468092411306, 13.100383032131553))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_2, 0.05, data_type="simple_frequency_distribution"), (6.329887815369333, 14.086552899795869))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_2, 0.02, data_type="simple_frequency_distribution"), (5.933036825242363, 15.359021371148184))));

𝜎^2 należy do przedziału [6.7683, 13.2330] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2 należy do przedziału [6.3940, 14.2292] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2 należy do przedziału [5.9931, 15.5145] przy założeniu poziomu ufności 1-𝛼 = 0.98

𝜎^2 należy do przedziału [6.7005, 13.1004] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2 należy do przedziału [6.3299, 14.0866] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2 należy do przedziału [5.9330, 15.3590] przy założeniu poziomu ufności 1-𝛼 = 0.98


In [14]:
### Komórka testowa

# Testy dla szeregów rozdzielczych przedziałowych

# Test na test_data_3 przy założeniu, że znane jest odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_variance_estimation(test_data_3, 0.1, population_mean=-5, data_type="grouped_frequency_distribution"), (3.9521130029725247, 6.320871976892474))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_3, 0.05, population_mean=-5, data_type="grouped_frequency_distribution"), (3.7921854749549446, 6.63839734543416))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_3, 0.02, population_mean=-5, data_type="grouped_frequency_distribution"), (3.6170094460586584, 7.034533746042385))));
print()

# Test na test_data_3 przy założeniu, że nie jest znane odchylenie standardowe populacji, z której została pobrana próbka losowa
assert(np.all(np.isclose(interval_variance_estimation(test_data_3, 0.1, data_type="grouped_frequency_distribution"), (3.940427083620819, 6.302181924229793))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_3, 0.05, data_type="grouped_frequency_distribution"), (3.7809724419283834, 6.618768408724867))));
print()
assert(np.all(np.isclose(interval_variance_estimation(test_data_3, 0.02, data_type="grouped_frequency_distribution"), (3.606314387326986, 7.013733481988378))));

𝜎^2 należy do przedziału [3.9521, 6.3209] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2 należy do przedziału [3.7922, 6.6384] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2 należy do przedziału [3.6170, 7.0345] przy założeniu poziomu ufności 1-𝛼 = 0.98

𝜎^2 należy do przedziału [3.9404, 6.3022] przy założeniu poziomu ufności 1-𝛼 = 0.9

𝜎^2 należy do przedziału [3.7810, 6.6188] przy założeniu poziomu ufności 1-𝛼 = 0.95

𝜎^2 należy do przedziału [3.6063, 7.0137] przy założeniu poziomu ufności 1-𝛼 = 0.98
